In [1]:
from __future__ import print_function
from functools import reduce
import re
from nltk.tokenize import word_tokenize
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent, Bidirectional, Dense, Flatten
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.models import load_model
import json
from pprint import pprint
import numpy as np
import csv
import string

Using TensorFlow backend.


In [2]:
def get_dictionary(vocab):
    d = dict()
    with open(vocab, 'r') as f:
        lines = f.readlines()
        for l in lines:
            w = l.strip().split()
            k = w[0]
            v = []
            for i in range(1, len(w)):
                v.append(float(w[i]))
            d[k] = np.array(v, dtype=np.float32)
    return d

In [3]:
d = get_dictionary("glove/glove.6B.100d.txt")

In [4]:
def tokenize_sequence(seq, max_length, dim):
    words = word_tokenize(seq.lower())
    word_vectors = list(map(lambda x: d[x] if x in d else np.zeros(dim), words))
    for i in range(len(word_vectors), max_length):
        word_vectors.append(np.zeros(dim))
    return np.array(word_vectors[:max_length])

In [5]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()
    
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [112]:
with open('test.json') as data_file:    
    data = json.load(data_file)

In [114]:
ids = []
titles = dict()
contexts = dict()
questions = dict()
answers_text = dict()
answers_start = dict()
for i in range(len(data)):
    paragraphs = data[i]["paragraphs"]
    title = data[i]["title"]
    for j in range(len(paragraphs)):
        context = paragraphs[j]["context"]
        qas = paragraphs[j]["qas"]
        for k in range(len(qas)):
#             answer = qas[k]["answer"]
            id_ = qas[k]["id"]
            question = qas[k]["question"]
            ids.append(id_)
            titles[id_] = title
            contexts[id_] = context
            questions[id_] = question
#             answers_start[id_] = answer["answer_start"]
#             answers_text[id_] = answer["text"]

In [60]:
max_para = 750
max_q = 50
dimension = 100

In [115]:
ind = [i for i in range(len(ids))]

In [116]:
paras = [tokenize_sequence(contexts[ids[ind[i]]],max_para,dimension) for i in range(len(ind))]

In [117]:
qns = [tokenize_sequence(questions[ids[ind[i]]],max_q,dimension) for i in range(len(ind))]

In [118]:
x = np.array(paras)

In [119]:
xq = np.array(qns)

In [14]:
model = load_model('model_9')

/Users/duynguyen/anaconda/lib/python3.5/site-packages/keras/models.py:287: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [195]:
out = model.predict([x,xq])

In [196]:
out_1 = np.argmax(out[0], axis=1)
out_2 = np.argmax(out[1], axis=1)

In [197]:
def search(para, pos):
    if pos == 0: 
        return 0
    if len(word_tokenize(para)) <= pos:
        return len(para)
    else:
        r = 0
        for i in range(1, len(para)):
            if len(word_tokenize(para[:i])) == pos:
                r = i
            if len(word_tokenize(para[:i])) > pos +1:
                break
        return r

In [198]:
for i in range(len(out[0])):
    s = 0
    t = 1
    if i%1000 == 0: 
        print(i)
    for j in range(750):
        for k in range(j+1, min(j+10, 750)):
            if out[0][i][s] * out[1][i][t] < out[0][i][j] * out[1][i][k]:
                s = j
                t = k
    out_1[i] = s
    out_2[i] = t
#     print(s, t)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000


In [202]:
out_2 = out_2 + 1

In [108]:
out_2

array([  2,  29,  16,  68,   2,  29,  44,  95,  83,   2,  84, 121,  29,
        14, 127, 127,  87,  87,  87, 127, 108, 127, 108,  87,   2,  53,
       127, 108,  37,  37,  37,  75,  37,  37,  37,  37,  32, 133, 106,
        37,  37,  63,  75,  16, 149, 139, 149, 125,  86, 125,  75,  63,
       106, 173, 125, 149,  45,  39,  52,  86,  62,  16,  62,  62,  52,
        78,  62,  86,  62,  67,  45,  54,  78,  67,  67,  78,  45, 120,
        76,   5,  79,  50,  45,  98,  78,  38,  16,  98,  38,  10,  89,
        38,  78,  10,  27,  27,  14,  15, 151,  15])

In [203]:
c = 0
for i in range(len(ind[:100])):
    
    a = search(contexts[ids[ind[i]]], out_1[i])
    b = search(contexts[ids[ind[i]]], out_2[i])
    if (a < b):
        print(ids[i])
        print(contexts[ids[ind[i]]])
        print("\n")
        print(questions[ids[ind[i]]])
        print("\n")
        print(normalize_answer(contexts[ids[ind[i]]][a:b]))
        print("\n")
#         print(normalize_answer(answers_text[ids[i]]))
        print("------------")
#         if normalize_answer(contexts[ids[ind[i]]][a:b]) == normalize_answer(answers_text[ids[i]]):
#             c += 1
print(c)

061379
Burke was born in Dublin, Ireland. His mother Mary née Nagle (c. 1702 – 1770) was a Roman Catholic who hailed from a déclassé County Cork family (and a cousin of Nano Nagle), whereas his father, a successful solicitor, Richard (died 1761), was a member of the Church of Ireland; it remains unclear whether this is the same Richard Burke who converted from Catholicism. The Burke dynasty descends from an Anglo-Norman knight surnamed de Burgh (latinised as de Burgo) who arrived in Ireland in 1185 following Henry II of England's 1171 invasion of Ireland.


Where was Burke born?


dublin ireland


------------
061380
Burke was born in Dublin, Ireland. His mother Mary née Nagle (c. 1702 – 1770) was a Roman Catholic who hailed from a déclassé County Cork family (and a cousin of Nano Nagle), whereas his father, a successful solicitor, Richard (died 1761), was a member of the Church of Ireland; it remains unclear whether this is the same Richard Burke who converted from Catholicism. The Bu

KeyboardInterrupt: 

In [ ]:
with open("submission_2.csv", 'w') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(['Id','Answer'])
    for i in ind:
        a = search(contexts[ids[i]], out_1[i])
        b = search(contexts[ids[i]], out_2[i])
        writer.writerow([ids[i],normalize_answer(contexts[ids[i]][a:b])])
        print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [194]:
search("don't blame",3)

11